#CFH - Customer Feedback Hub
##Fonctions associées aux appels des cognitive services Azure

In [0]:
print('#'*80)
import requests, uuid, datetime
import pandas as pd
import numpy as np
from io import StringIO, BytesIO

print('-'*80)
print('Starting - ' + str(datetime.datetime.now()))
print('-'*80)


## API Traduction
def GetTranslations (text_to_translate, subscriptionKey, region, constructed_url ):
    # Recuperer la traduction en anglais (par defaut)
	headers = {
    'Ocp-Apim-Subscription-Key': subscriptionKey,
    'Ocp-Apim-Subscription-Region' : region,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
    }

	body = [{ 'text': text_to_translate}]
	request = requests.post(constructed_url, headers=headers, json=body)
	response = request.json()    

	return response


## API Analysis

def GetKeyPrhase (text_to_analyse, lang , subscriptionKey, region, constructed_url ):
    # Extractions des groupes nominaux principaux
	headers = {
    'Ocp-Apim-Subscription-Key': subscriptionKey,
    'Ocp-Apim-Subscription-Region' : region,
    'Content-type': 'application/json',
    'Accept' : 'application/json'    }
	body = {'documents' : [{ 'id': '1', 'language' : lang, 'text' : text_to_analyse}] }
	request = requests.post(constructed_url+'/keyPhrases', headers=headers, json=body)
	response = request.json()    
    
	return response

def GetSentiment (text_to_analyse, lang , subscriptionKey, region, constructed_url ):
    # Extraction du sentiment du commentaire
	headers = {
    'Ocp-Apim-Subscription-Key': subscriptionKey,
    'Ocp-Apim-Subscription-Region' : region,
    'Content-type': 'application/json',
    'Accept' : 'application/json'    }
	body = {'documents' : [{ 'id': '1', 'language' : lang, 'text' : text_to_analyse}] }
	request = requests.post(constructed_url+'/sentiment', headers=headers, json=body)
	response = request.json()    
    
	return response


## API Langage Understanding

def GetLuis(text_to_analyse, subscriptionKey, constructed_url ):
    #cette fonction permet de classifier un texte -- nouvelle version
    param = {'verbose' : 'true',
             'subscription-key' : subscriptionKey,
             'show-all-intents' : 'true',
             'log' : 'true',
             'query' : text_to_analyse}
    request = requests.get(constructed_url, params = param)
    response = request.json()        
    return response

def LUIS_CFH(lang, text, key,apps_id):
    #set on the good luis according to language
    if len(text)>500:
        return {'query' : text,
                'prediction' :  {'topIntent' : 'test too long, not supported ',
                                'intents' : {'None': {'score': 1},
                                             'General POSITIVE comment': {'score': 0.},
                                             'EV Charging': {'score': 0.},
                                             'Lubricants': {'score': 0.},
                                             'Total Assistance': {'score': 0.},
                                             'Pellets': {'score': 0.},
                                             'Fuel Quality': {'score': 0.},
                                             'Toilets': {'score': 0.},
                                             'Safety': {'score': 0.},
                                             'General NEGATIVE comment': {'score': 0.},
                                             'Fleet Cards': {'score': 0.},
                                             'Waiting Time': {'score': 0.},
                                             'Coffee and Sandwiches': {'score': 0.},
                                             'Total wash': {'score': 0.},
                                             'Heating Fuel': {'score': 0.},
                                             'Price': {'score': 0.},
                                             'LPG/CNG': {'score': 0.},
                                             'AdBlue': {'score': 0.},
                                             'Offers': {'score': 0.},
                                             'Total Club': {'score': 0.},
                                             'Cleanliness': {'score': 0.},
                                             'Welcome': {'score': 0.}},
                                             'entities': {}}}
    elif lang =='fr':  
        url = 'https://westeurope.api.cognitive.microsoft.com/luis/prediction/v3.0/apps/'+apps_id[lang]+'/slots/production/predict'
        return GetLuis(text,key,url)
    elif lang == 'nl':
         url = 'https://westeurope.api.cognitive.microsoft.com/luis/prediction/v3.0/apps/'+apps_id[lang]+'/slots/production/predict'
         return GetLuis(text,key,url)
    elif lang == 'en':
         url = 'https://westeurope.api.cognitive.microsoft.com/luis/prediction/v3.0/apps/'+apps_id[lang]+'/slots/production/predict'
         return GetLuis(text,key,url)
    else:
        return {'query' : text,
                'prediction' :  {'topIntent' : 'language not supported',
                                'intents' : {'None': {'score': 1},
                                             'General POSITIVE comment': {'score': 0.},
                                             'EV Charging': {'score': 0.},
                                             'Lubricants': {'score': 0.},
                                             'Total Assistance': {'score': 0.},
                                             'Pellets': {'score': 0.},
                                             'Fuel Quality': {'score': 0.},
                                             'Toilets': {'score': 0.},
                                             'Safety': {'score': 0.},
                                             'General NEGATIVE comment': {'score': 0.},
                                             'Fleet Cards': {'score': 0.},
                                             'Waiting Time': {'score': 0.},
                                             'Coffee and Sandwiches': {'score': 0.},
                                             'Total wash': {'score': 0.},
                                             'Heating Fuel': {'score': 0.},
                                             'Price': {'score': 0.},
                                             'LPG/CNG': {'score': 0.},
                                             'AdBlue': {'score': 0.},
                                             'Offers': {'score': 0.},
                                             'Total Club': {'score': 0.},
                                             'Cleanliness': {'score': 0.},
                                             'Welcome': {'score': 0.}},
                                             'entities': {}}}



def GetSuggestedAnswer(lang, sentiment, prio, connect):
    #cette fonction permet de selectionner la bonne reponse suggeree 
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connect['string'])
        blob_client = blob_service_client.get_blob_client(container=connect['container'], blob="suggested_answer.csv")
        my_string = str( blob_client.download_blob().readall() ,'latin-1')
        data = StringIO(my_string) 
        df = pd.read_csv(data, sep = ";", decimal = ",")  
        return str(df.suggested.loc[(df.language == lang) & (df.sentiment_min <=  sentiment) & (df.sentiment_max > sentiment) & (df.topic == prio)].iloc[0])
    except Exception as ex:
        return "Exceptation - " + str(ex)

# fonctions -------------------------------------------------------------------

def GetTranslations_multi (data_id_text, subscriptionKey, region, constructed_url ):
    #recuperer la langue et la traduction de ZCL par batch
	headers = {'Ocp-Apim-Subscription-Key': subscriptionKey,
               'Ocp-Apim-Subscription-Region' : region,
                'Content-type': 'application/json'    }
	body = data_id_text[["id","text"]].to_dict(orient= "reccords")

	request = requests.post(constructed_url, headers=headers, json=body)
	response = request.json()   
    
	malist = []
	for item in response:
		malangue = str(item['detectedLanguage']['language'])  #str= error mgmnt
		matrad = str(item['translations'][0]['text'])         #str= error mgmnt
		malist.append({'language' : malangue, 'translation' : matrad})
		del [matrad,malangue]
        
	return pd.DataFrame(malist)

def GetKeyPrhase_multi (id_lang_text_df, subscriptionKey, region, constructed_url ):
    #recuperer les KeyPhrasees de ZCL par batch
	headers = {'Ocp-Apim-Subscription-Key': subscriptionKey,
               'Ocp-Apim-Subscription-Region' : region,
               'Content-type': 'application/json',
               'Accept' : 'application/json'    }
	body = {'documents' : id_lang_text_df.to_dict(orient= "reccords") }
	request = requests.post(constructed_url+'/keyPhrases',
                         headers=headers, 
                         json=body)
	response = request.json()        
	return response

def GetSentiment_multi (id_lang_text_df, subscriptionKey, region, constructed_url ):
     #recuperer les KeyPhrasees de ZCL par bactch
	headers = {'Ocp-Apim-Subscription-Key': subscriptionKey,
               'Ocp-Apim-Subscription-Region' : region,
               'Content-type': 'application/json',
               'Accept' : 'application/json'}
	body = {'documents' : id_lang_text_df.to_dict(orient= "reccords") }
	request = requests.post(constructed_url+'/sentiment', 
                         headers=headers, 
                         json=body)
	response = request.json()        
	return response


def GetSuggested_multi(df_in, connect):
    #recuperer les reponse preformattee par bactch
    blob_service_client = BlobServiceClient.from_connection_string(connect['string'])
    blob_client = blob_service_client.get_blob_client(container=connect['container'], blob="suggested_answer.csv")
    my_string = str( blob_client.download_blob().readall() ,'latin-1')
    data = StringIO(my_string) 
    df = pd.read_csv(data, sep = ";", decimal = ",")  

    df_out = df_in.merge(df, 
                             left_on = ['language','Priorite'],
                             right_on = ['language','topic'],
                             how = 'left')
    df_out = df_out.loc[((df_out.sentiment_min < df_out.Sentiment)&(df_out.sentiment_max > df_out.Sentiment)) | 
            (df_out.Sentiment.isna() ) |
            (df_out.Priorite.isna()  ) |
            (~df_out.language.isin(['en','fr','nl']))]

    del [ df_out['sentiment_min'], df_out['sentiment_max'], df_out['topic'] ]
    df_out = df_out.drop_duplicates(['id'], keep= 'first')
    
    return df_out

# fonction consolidee ---------------------------------------------------------
    
def TheVoice_multi(df_in,
                   Key_TT, reg_TT, Url_TT,
                   Key_TA, reg_TA, Url_TA,
                   Key_LU, apps_id,
                   connect):
    #analyse d'un df sur l'ensemble des appels 
    #verif du fichier reçu
    try:
        df = df_in[["id","text"]]
        df = df.loc[ ~ df.text.isin( ['nan', 'NaN', np.nan, np.NaN])]
        df = df.loc[df.text.apply(len)>5]
    except Exception as ex:
        return "DataFrame format error, need id and text cols - " + str(ex)
    #traduction
    try :
        temp = GetTranslations_multi(df, Key_TT, reg_TT, Url_TT)
        df_out = pd.concat([df.reset_index(),temp], axis = 1)
        df_out['analysed_lang'] = 'en'
        del temp
    except Exception as ex:
        return "Translation Error - " + str(ex)
    #KeyPhrase
    try:
        temp = GetKeyPrhase_multi(df_out[["id","analysed_lang","translation"]].rename(
                columns={'translation':'text','analysed_lang':'language'}),
                        Key_TA, reg_TA, Url_TA )
        df_out = pd.concat([df_out,
                            pd.DataFrame(temp['documents'])['keyPhrases']], 
                            axis = 1)
        del temp
    except Exception as ex:
        return "KeyPhrase Error - " + str(ex)
    #Sentiment
    try:
        temp = GetSentiment_multi(df_out[["id","language","text"]],
                        Key_TA , reg_TA, Url_TA)
        df_out = pd.concat([df_out,
                            pd.DataFrame(temp['documents'])['score']], 
                            axis = 1)
        del temp
    except Exception as ex:
        return "Sentiment Error - " + str(ex)
    #Reponse
    try:
        df_out["Priorite"] = ''
        df_out["Priorite score"] = np.nan
        df_out["LUIS"] = ''
        for i in range(len(df_out)):
            try:
                temp = LUIS_CFH(df_out.language.iloc[i], df_out.text.iloc[i], Key_LU, apps_id)
            except Exception as ex:
                print('LUIS query error' + str(ex) )
                
            temp = eval("[{'intent':"+str(temp['prediction']['intents'])[1:-2].replace(": {'score':",", 'score' : ").replace("},","},{'intent':")+"}]")
            df_out["LUIS"].iloc[i] = str(temp)
            df_out["Priorite"].iloc[i] = temp[0]['intent']
            df_out["Priorite score"].iloc[i] = temp[0]['score']
    except Exception as ex:
        return "LUIS process error - " + str(ex) + ' - ' + str(i) + ' - ' + str(temp)
    
    df_out = GetSuggested_multi(df_out, connect)   
    
    del [df_out['text'], df_out['index'] ]
    return df_in.merge(df_out.rename(columns={'score' : 'Sentiment'}), left_on = 'id', right_on = 'id', how = 'left' ).copy()


def TheVoice_multi_lines(df_in,
                   Key_TT, reg_TT, Url_TT,
                   Key_TA, reg_TA, Url_TA,
                   Key_LU, apps_id,
                   connect):
    #analyse ligne par ligne d'un df sur l'ensemble des appls
    #verif du fichier reçu
    try:
        df = df_in[["id","text"]]
        df = df.loc[ ~ df.text.isin( ['nan', 'NaN', np.nan, np.NaN])]
        df = df.loc[df.text.apply(len)>5]
    except Exception as ex:
        return "DataFrame format error, need id and text cols - " + str(ex)
    #traduction
    df['language'] =np.nan
    df['translation'] = np.nan
    df['analysed_lang'] = 'en'
    df['keyPhrases'] = np.nan
    df['Sentiment'] = np.nan
    df["Priorite"] = ''
    df["Priorite score"] = np.nan
    df["LUIS"] = ''
    for i in range(len(df)):
        try:
            temp = GetTranslations(df.text.iloc[i] ,Key_TT, reg_TT, Url_TT)[0]
            df["language"].iloc[i]=str(temp['detectedLanguage']['language'])
            df["translation"].iloc[i]=str(temp['translations'][0]['text'])
            del temp
        except Exception as ex:
            print( "Translation lines error - " + str(ex) + " line : " + str(i))
        
        try:
            df["keyPhrases"].iloc[i] = GetKeyPrhase(df.translation.iloc[i],
                              df.analysed_lang.iloc[i],
                              Key_TA, reg_TA,
                              Url_TA)['documents'][0]['keyPhrases']
        except Exception as ex:
            print( "KeyPhrases lines error - " + str(ex) + " line : " + str(i))
        
        try:
            df["Sentiment"].iloc[i] = GetSentiment(df.text.iloc[i],
                                      df.language.iloc[i],
                                      Key_TA, reg_TA,
                                      Url_TA)['documents'][0]['score']
        except Exception as ex:
            print( "Sentiment lines error - " + str(ex) + " line : " + str(i))


        try:
            temp = LUIS_CFH(df.language.iloc[i], df.text.iloc[i][:495], Key_LU, apps_id)
        except Exception as ex:
                print('LUIS query error' + str(ex) )
        try:        
            temp = eval("[{'intent':"+str(temp['prediction']['intents'])[1:-2].replace(": {'score':",", 'score' : ").replace("},","},{'intent':")+"}]")
            df["LUIS"].iloc[i] = str(temp)
            df["Priorite"].iloc[i] = temp[0]['intent']
            df["Priorite score"].iloc[i] = temp[0]['score']
            del temp
        except Exception as ex:
            return "LUIS process error - " + str(ex) + ' - ' + str(i)
    
    df = GetSuggested_multi(df, connect)   
    
    del [df['text'] ]
    return df_in.merge(df.rename(columns={'score' : 'Sentiment'}), left_on = 'id', right_on = 'id', how = 'left' ).copy()


print('#'*80)